In [40]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from lazypredict.Supervised import LazyRegressor 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import PoissonRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
salarios =  pd.read_csv('data/salaries_data.csv')
testeo =  pd.read_csv('data/testeo.csv')
muestra = pd.read_csv('data/muestra.csv')

In [3]:
salarios.shape

(500, 11)

In [4]:
testeo.shape

(107, 8)

In [5]:
salaries = salarios.salary_in_usd

In [6]:
salarios.drop(columns = ['salary', 'salary_currency', 'salary_in_usd'], inplace = True)

In [7]:
salarios.info(), testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           500 non-null    int64 
 1   experience_level    500 non-null    object
 2   employment_type     500 non-null    object
 3   job_title           500 non-null    object
 4   employee_residence  500 non-null    object
 5   remote_ratio        500 non-null    int64 
 6   company_location    500 non-null    object
 7   company_size        500 non-null    object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           107 non-null    int64 
 1   experience_level    107 non-null    object
 2   employment_type     107 non-null    object
 3   job_title       

(None, None)

In [8]:
concat = pd.concat([salarios, testeo])

In [9]:
concat.experience_level.value_counts()

SE    280
MI    213
EN     88
EX     26
Name: experience_level, dtype: int64

Siginifcado de cada variable y número asignado a cada una:

+ EN: entry level -> 1
+ MI: mid - senior -> 2
+ SE: senior -> 3
+ EX: executive -> 4

In [10]:
exp_level = {'EN': 1, 'MI': 2, 'SE': 3, 'EX': 4}
experience_level = []
for e in concat.experience_level:
    if e in exp_level:
        experience_level.append(exp_level[e])

In [11]:
concat['experience_level'] = experience_level

In [12]:
concat.employment_type.value_counts()

FT    588
PT     10
CT      5
FL      4
Name: employment_type, dtype: int64

Siginifcado de cada variable y número asignado a cada una:

+ FT: full-time -> 4
+ PT: part-time -> 3
+ CT: contract -> 2
+ FL: freelance -> 1

In [13]:
emp_type = {'FL': 1, 'CT': 2, 'PT': 3, 'FT': 4}
employment_type = []
for e in concat.employment_type:
    if e in emp_type:
        employment_type.append(emp_type[e])

In [14]:
concat['employment_type'] = employment_type

In [15]:
concat.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,2022,3,4,Data Engineer,US,100,US,M
1,2022,3,4,Data Engineer,US,100,US,M
2,2021,2,4,BI Data Analyst,US,100,US,M
3,2021,2,2,ML Engineer,US,100,US,L
4,2021,2,4,Data Engineer,RO,0,US,L


In [16]:
concat.shape

(607, 8)

In [17]:
le = LabelEncoder()
concat['company_location'] = le.fit_transform(concat.company_location)
concat['job_title'] = le.fit_transform(concat.job_title)

In [18]:
comp_size = {'L': 3, 'M': 2, 'S': 1}
company_size = []
for e in concat.company_size:
    if e in comp_size:
        company_size.append(comp_size[e])

concat['company_size'] = company_size

In [19]:
concat.drop(columns = ['employee_residence'], inplace = True)

In [20]:
sal = concat.iloc[:500, :]
test = concat.iloc[500:, :]

In [21]:
sal['salaries'] = salaries

### Utilizamos LazyRegressor() para saber cuál es el modelo de Machine Learning que mejor nos puede funcionar a la hora de predecir los salarios

In [35]:
X = sal.drop(columns = ['salaries'])

y = sal.salaries

In [36]:
X_train, X_test, y_train, y_test = tts(X, y, train_size = 0.8)

In [37]:
lazy=LazyRegressor()

In [39]:
lazy.fit(X_train, X_test, y_train, y_test)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:03<00:00, 11.61it/s]


(                               Adjusted R-Squared  R-Squared       RMSE  \
 Model                                                                     
 HistGradientBoostingRegressor                0.55       0.58   44272.08   
 LGBMRegressor                                0.53       0.57   44999.59   
 GradientBoostingRegressor                    0.45       0.49   48896.20   
 RandomForestRegressor                        0.44       0.48   49484.27   
 BaggingRegressor                             0.41       0.45   50807.02   
 PoissonRegressor                             0.39       0.44   51348.77   
 SGDRegressor                                 0.38       0.42   52113.33   
 LassoLars                                    0.38       0.42   52118.47   
 Lasso                                        0.38       0.42   52118.51   
 Lars                                         0.38       0.42   52118.52   
 TransformedTargetRegressor                   0.38       0.42   52118.52   
 LinearRegre

In [40]:
PR = PoissonRegressor()

In [30]:
PR.fit(X, y)

PoissonRegressor()

In [31]:
pred = PR.predict(test)

In [33]:
muestra['salary_in_usd'] = pred

In [35]:
muestra.to_csv('prediction_models/muestra(Poisson).csv', index = False)

In [45]:
LGBM = LGBMRegressor()

In [46]:
LGBM.fit(X, y)

LGBMRegressor()

In [47]:
pred2 = LGBM.predict(test)

In [48]:
muestra['salary_in_usd'] = pred2

In [49]:
muestra.to_csv('prediction_models/muestra(LGBM).csv', index = False)

In [29]:
GBR = GradientBoostingRegressor()

In [30]:
GBR.fit(X,y)

GradientBoostingRegressor()

In [32]:
pred3 = GBR.predict(test)

In [33]:
muestra['salary_in_usd'] = pred3

In [47]:
muestra.to_csv('prediction_models/muestra(GBR2).csv', index = False)

In [41]:
HGBR = HistGradientBoostingRegressor()

In [42]:
HGBR.fit(X,y)

HistGradientBoostingRegressor()

In [44]:
pred4 = HGBR.predict(test)

In [45]:
muestra['salary_in_usd'] = pred4

In [48]:
muestra.to_csv('prediction_models/muestra(HGBR).csv', index = False)